# Coverage Report Generation V2.0

## 1. Variables to modify

In [1]:
user = 'jmarinbe@cisco.com' # Your Cisco e-mail address
cr_tableau_template_name = 'C:/Users/jmarinbe/OneDrive - Cisco/Cisco Tableau/CR Template.twb' # twb template name/path
personal_access_token_name = 'juanma'
personal_access_token_secret = 'Sq1X69ihR+Wy+CgkQQCVYQ==:2e30VjYlJu9SZp0goiTZh34coeC9Avk6'

## 2. Libraries & Functions

In [2]:
%%time
# ----------------------------------------- LIBRARIES

#!pip install snowflake-connector-python
#!pip install tableauhyperapi
#!pip install tableau-api-lib
#!pip install tableauserverclient
#!pip install hyperapi
#!pip install pyarrow
#!pip install fastparquet

import os
import datetime
import pandas as pd
import numpy as np
import itertools
import shutil
import xml.etree.ElementTree as ET
import snowflake.connector
import re
import webbrowser

from zipfile import ZipFile
import tableauserverclient as TSC
from tableauhyperapi import *

import smartsheet_lib as smartsheet
import utils_coverage as utilsc

# ----------------------------------------- FUNCTIONS

qs = {8:1,9:1,10:1,11:2,12:2,1:2,2:3,3:3,4:3,5:4,6:4,7:4}
fy = {1:1,2:1,3:0,4:0}

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

Wall time: 1.13 s


## 3. Generating the Workbooks

In [3]:
%%time
    
current_path = os.getcwd().replace("\\","/")

date = datetime.datetime.today()
date = date.date()

if date.weekday() in [0,1]:
    delta_t = str(date-datetime.timedelta(4))
else:
    delta_t = str(date-datetime.timedelta(1))
    
month = datetime.datetime.today().strftime("%B")

print ('Current Path: ' + str(current_path))
print ('Current Date: ' + str(date))

# -----------------------------------------------------------------

smartsheet_client = smartsheet.init_conn("J937aTx1hHKkv3ju15G0XGC0Qz6af50MNrhLq")
oa_sheet = smartsheet.load_sheet(7190965472520068,client=smartsheet_client,modified_since=delta_t)
oa_df = pd.DataFrame()
sheet_new = smartsheet.get_last_n_rows(oa_sheet,n_rows=3000)
oa_df = smartsheet.sheet_to_df2(sheet_new,columns=oa_sheet.columns)
oa_df = oa_df.query("`Request ID` != ''")
oa_df['Request ID'] = oa_df['Request ID'].apply(lambda x:int(x)) # drop request id decimal places

fields_df = utilsc.get_da_requests(da=user,df=oa_df)
fields_df['CR Creation Date'] = date
fields_df

Current Path: C:/Users/jmarinbe/OneDrive - Cisco/Cisco Tableau
Current Date: 2022-03-11
Loaded Sheet: OP Intake Request Fields FY22Q3
Rows: 413
Wall time: 10.1 s


,index,Request ID,Date Created,Assigned DA,Campaign Name,Customer Name,Input file URL,ID TYPE,SAV ID,CAV ID,...,Cov Rejected Reason,Cov URL,Cov Hold Reason,Cov Uploaded Time,sav_list,gu_list,cav_list,cr_list,contract_list,CR Creation Date
0,0,112194,2022-02-16 18:16:00,jmarinbe@cisco.com,Coverage Program - SNTC Proactive Attach,BRITISH TELECOM,,SAV ID,283604009,,...,,,,,[283604009],[97073],[],"[5959819, 300730365, 270092162, 32744770, 2694...",[],2022-03-11
1,1,112195,2022-02-16 18:16:00,jmarinbe@cisco.com,Coverage Program - SNTC Proactive Attach,SYSTEMS CDS GMBH_ non TA,,SAV ID,203896342,,...,,,,,[203896342],[97073],[],"[216257990, 89102066]",[],2022-03-11
2,2,112196,2022-02-16 18:16:00,jmarinbe@cisco.com,Coverage Program - SNTC Proactive Attach,LIBERTY GLOBAL VIRGIN,,SAV ID,203750180,,...,,,,,[203750180],[97073],[],[],[],2022-03-11
3,3,112197,2022-02-16 18:16:00,jmarinbe@cisco.com,Coverage Program - SNTC Proactive Attach,DEUTSCHE TELEKOM AG,,SAV ID,203722826,,...,,,,,[203722826],[97073],[],[],[],2022-03-11
4,4,112198,2022-02-16 18:16:00,jmarinbe@cisco.com,Coverage Program - SNTC Proactive Attach,DEUTSCHE TELEKOM AG,,SAV ID,227847508,,...,,,,,[227847508],[97073],[],[],[],2022-03-11
5,5,112199,2022-02-16 18:16:00,jmarinbe@cisco.com,Coverage Program - SNTC Proactive Attach,DEUTSCHE_ TELEKOM_ AG,,SAV ID,203855481,,...,,,,,[203855481],[97073],[],[],[],2022-03-11
6,6,112200,2022-02-16 18:16:00,jmarinbe@cisco.com,Coverage Program - SNTC Proactive Attach,DEUTSCHE TELEKOM AUSTRIA,,SAV ID,203895606,,...,,,,,[203895606],[97073],[],[],[],2022-03-11


In [4]:
%%time
# Retrieving snowflake data and creating dataframes
savs_list,gu_list,cav_list,cr_list = utilsc.get_ids_list(fields_df,separator=',') # IDs to look for on Snowflake
uncovered_data = utilsc.get_uncovered_data(user,ids_sav=savs_list,ids_gu=gu_list,ids_cr=cr_list,ids_cav=cav_list)  # Retrieve Uncovered Data from SF
coverage_data = utilsc.get_coverage_data(user,ids_sav=savs_list,ids_gu=gu_list,ids_cr=cr_list,ids_cav=cav_list) # Retrieve Coverage Data from SF
contracts_data = utilsc.get_contracts_data(user,ids_sav=savs_list,ids_gu=gu_list,ids_cr=cr_list,ids_cav=cav_list)  # Retrieve Contracts Data from SF
df_uncovered,df_coverage,df_contracts = utilsc.format_columns(uncovered_data,coverage_data,contracts_data) # Rename columns to match tableau extracts

Wall time: 46.5 s


In [5]:
%%time
sns_df = pd.DataFrame()
folders_list = []
smartsheet_fields = ['index','Request ID','Date Created',
                         'Assigned DA','Campaign Name','Customer Name',
                         'Input file URL','ID TYPE','SAV ID','CAV ID',
                         'CAV BU ID','GU ID','Lvl1','Lvl2 (Region)',
                         'Contract ID','Inventory Name','Appliance ID',
                         'CR Party Name','CR Party ID','Comments','DA Comments',
                         'Status','Requester Name','Who should be notified on completion of Analysis',
                         'OP Status','CR Creation Date']
for idx in range(len(fields_df)):
    name = re.sub('[^A-Za-z0-9\-]+', '', fields_df['Customer Name'][idx][0:15])
    theater = fields_df['Lvl1'][idx]
    req_type = fields_df['ID TYPE'][idx]
    
    if req_type == 'SAV ID':
        ids = fields_df['sav_list'][idx]
        id_type = 'SAV'
    elif req_type == 'GU ID':
        ids = fields_df['gu_list'][idx]
        id_type = 'GU'
    elif req_type == 'CR Party ID':
        ids = fields_df['cr_list'][idx]
        id_type = 'CR'
    elif req_type == 'CAV ID':
        ids = fields_df['cav_list'][idx]
        id_type = 'CAV'
        
    req_id = fields_df['Request ID'][idx]
    date_created = fields_df['CR Creation Date'][idx]
    print(f"{idx+1}/{len(fields_df)}: {name}-{req_id}",end=" - ")
    
    folder =  f"CR_{theater}_{req_id}_{name}_{req_type}_{str(date.year)}_{str(date.month)}_{str(date.day)}" # folder name
    folder_path = f"{current_path}/CR/{month}/{date}/{folder}"    
      
    # -------------------------------------------------- Creating the extracts   
    uncovered_data_filtered = df_uncovered[df_uncovered['Bk Sales Account Id Int'].isin([int(customer_id) for customer_id in ids])]
    uncovered_data_filtered = uncovered_data_filtered[uncovered_data_filtered['ACCOUNT_IDENTIFIER'] == id_type]
    if len(uncovered_data_filtered) == 0:
        print(bcolors.FAIL + f"No Uncovered Oppty - No Serial Numbers to upload" + bcolors.ENDC)
        folders_list.append("N/A")
        
    else:
        try:
            os.makedirs(f"{folder_path}/Data")
        except: pass    
        folders_list.append(folder)
        
        coverage_data_filtered = df_coverage[df_coverage['Bk Sales Account Id Int'].isin([int(customer_id) for customer_id in ids])]
        coverage_data_filtered = coverage_data_filtered[coverage_data_filtered['ACCOUNT_IDENTIFIER'] == id_type]
        contracts_data_filtered = df_contracts[contracts_data['Bk Sales Account Id Int'].isin([int(customer_id) for customer_id in ids])]
        contracts_data_filtered = contracts_data_filtered[contracts_data_filtered['ACCOUNT_IDENTIFIER'] == id_type]
        smartsheet_filtered = fields_df[smartsheet_fields].iloc[[idx]]
        
        # Removing some columns
        uncovered_data_filtered.drop(['ACCOUNT_IDENTIFIER'],axis = 1,inplace = True)
        coverage_data_filtered.drop(['ACCOUNT_IDENTIFIER'],axis = 1,inplace = True) 
        contracts_data_filtered.drop(['ACCOUNT_IDENTIFIER'],axis = 1,inplace = True)   

        uncovered_data_filtered.set_index('Bk Sales Account Id Int').to_csv(f"{folder_path}/Data/uncovered.csv")
        coverage_data_filtered.set_index('Bk Sales Account Id Int').to_csv(f"{folder_path}/Data/contracts-and-coverage.csv")
        contracts_data_filtered.set_index('Bk Sales Account Id Int').to_csv(f"{folder_path}/Data/contract-view.csv")
        smartsheet_filtered.to_csv(f"{folder_path}/Data/Smartsheet.csv")
        

        # -------------------------------------------------- Creating the workbooks twb
        """tree = ET.parse(cr_tableau_template_name)
        for extract_path in tree.getroot().findall("./datasources/datasource/connection/named-connections/named-connection/connection[@class='textscan']"):
            extract_name = extract_path.attrib['directory']#.split('/')[-1]
            extract_path.attrib['directory'] = f"{folder_path}/Data"

        try:
            with open (f"{folder_path}/{folder}.twb", "wb") as files :                                             
                tree.write(files)

            utilsc.convert_to_twbx(f"{folder_path}")
            print("twbx created successfully")

            # -------------------------------------------------- Getting Serial Numbers to upload to SF
            sn_df = uncovered_data_filtered[['Serial Number','Bk Sales Account Id Int','Branch Primary Name']]
            sn_df['USER_ID'] = user
            sn_df['SALES_LEVEL_1_NAME'] = theater
            sn_df['SALES_LEVEL_2_NAME'] = fields_df['Lvl2 (Region)'][idx]
            sn_df['COMPASS_REQ_ID'] = req_id
            sn_df['COMPLETION_QUARTER'] = f"FY{date_created.year + fy.get(qs.get(date_created.month))}Q{qs.get(date_created.month)}"
            sn_df['COMPLETION_DATE'] = date_created
            sn_df.rename(columns={'Serial Number':'SERIAL_NUMBER','Bk Sales Account Id Int':'CUSTOMER_ID','Branch Primary Name':'CUSTOMER_NAME'},inplace=True)
            sn_df["COMPLETION_DATE"] = sn_df["COMPLETION_DATE"].astype(str)
            sn_df['ACCOUNT_IDENTIFIER'] = id_type
            sn_df = sn_df[['USER_ID','SERIAL_NUMBER','CUSTOMER_ID','CUSTOMER_NAME','ACCOUNT_IDENTIFIER','SALES_LEVEL_1_NAME','SALES_LEVEL_2_NAME','COMPASS_REQ_ID','COMPLETION_QUARTER','COMPLETION_DATE']]
            sns_df = pd.concat([sns_df,sn_df])                
        except: print(bcolors.FAIL + "Error" + bcolors.ENDC) """

1/7: BRITISHTELECOM-112194 - 2/7: SYSTEMSCDSGMB-112195 - 3/7: LIBERTYGLOBAL-112196 - 4/7: DEUTSCHETELEKO-112197 - 5/7: DEUTSCHETELEKO-112198 - 6/7: DEUTSCHETELEK-112199 - 7/7: DEUTSCHETELEKO-112200 - Wall time: 4.5 s
